## Задание

Разобраться с моделькой генерации текста, собрать самим или взять датасет с вебинара и обучить генератор.

---

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import time

import warnings
warnings.filterwarnings('ignore')

In [ ]:
text = open('evgenyi_onegin.txt','rb').read().decode(encoding = 'utf-8')
print('Length of text: {} characters'.format(len(text)))

In [ ]:
text = text + text

In [ ]:
# Remove duplicates

vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

In [ ]:
# Text as int

char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

---

In [ ]:
# Train and target

seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder = True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

---

In [ ]:
# Batch and buffer

BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder = True)

In [ ]:
vocab_size = len(vocab)
embedding_dim = 128
rnn_units = 1024

In [ ]:
# class RNNgenerator

class RNNgenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super(RNNgenerator, self).__init__()
        
        self.emb = tf.keras.layers.Embedding(vocab_size, embedding_dim)
                                 
        self.gru1 = tf.keras.layers.GRU(rnn_units,
                            return_sequences = True,
                            stateful = False,
                            recurrent_initializer = 'glorot_uniform')

        self.gru2 = tf.keras.layers.GRU(rnn_units,
                            return_sequences = True,
                            stateful = False,
                            recurrent_initializer = 'glorot_uniform')
        self.gru3 = tf.keras.layers.GRU(rnn_units,
                            return_sequences = True,
                            stateful = False,
                            recurrent_initializer = 'glorot_uniform')

        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        emb_x = self.emb(x)
        x = self.gru1(emb_x)
        x = self.gru2(x)
        x = self.gru3(x)

        x = self.dense(x)
        return x 

model = RNNgenerator(vocab_size,
                     embedding_dim,
                     rnn_units = rnn_units
                     )

---

In [ ]:
# def loss

def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits = True)

In [ ]:
model.compile(optimizer = 'adam', loss = loss, metrics = 'accuracy')

In [ ]:
# Checkpoints

checkpoint_dir = './RNN_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                                filepath = checkpoint_prefix,
                                save_freq = 5,
                                save_weights_only = True)

---

In [ ]:
# Training

EPOCHS = 500
history = model.fit(dataset,
                    epochs = EPOCHS,
                    callbacks = [checkpoint_callback])

In [ ]:
plt.plot(history.history['loss'])
plt.show()

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

---

In [ ]:
model = RNNgenerator(vocab_size, embedding_dim, rnn_units)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

---

In [ ]:
# Generate text

num_generate = 500
temperature = 1

In [ ]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [ ]:
text_ = generate_text(model, start_string = u"И вот идет уже ")
print(text_)

In [ ]:
# Check 200 epoch

model = RNNgenerator(vocab_size, embedding_dim, rnn_units) 
model.load_weights('./RNN_checkpoints/ckpt_200')
model.build(tf.TensorShape([1, None]))

In [ ]:
text_ = generate_text(model, start_string = u"И вот идет уже ")
print(text_)

---

In [ ]:
class RNNgenerator_1(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru1 = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.gru2 = tf.keras.layers.GRU(rnn_units,
                                    return_sequences=True,
                                    return_state=True,
                                    recurrent_initializer='glorot_uniform')
    self.gru3 = tf.keras.layers.GRU(rnn_units,
                                    return_sequences=True,
                                    return_state=True,
                                    recurrent_initializer='glorot_uniform')
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, x, states=None, return_state=False):

    x = self.embedding(x)

    if states is None:
      states = self.gru1.get_initial_state(x)

    x, states = self.gru1(x, initial_state=states)
    x, states = self.gru2(x, initial_state=states)
    x, states = self.gru3(x, initial_state=states)

    x = self.dense(x)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
heckpoint_dir = './RNN_checkpoints'

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                                filepath = checkpoint_prefix,
                                save_freq = 1,
                                save_weights_only = True)

In [ ]:
model_1 = RNNgenerator_1(
                 vocab_size = vocab_size,
                 embedding_dim = embedding_dim,
                 rnn_units = rnn_units,
                )

In [ ]:
model_1.compile(optimizer = 'adam', loss = loss, metrics = 'accuracy')

In [ ]:
model_1.load_weights('./RNN_checkpoints/ckpt_200')

---

In [ ]:
# Generate text

def generate_text_1(model, start_string, states, temperature):
    # Evaluation step (generating text using the learned model)

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions, states = model(input_eval, states=states, return_state=True)
        predictions = tf.squeeze(predictions, 0)
        
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [ ]:
num_generate = 500
temperature = 0.6
states = None
next_char = u"И вот идет уже "
text_generated = []

In [ ]:
text = generate_text_1(model_1, next_char, states, temperature)
print(text)

---

In [ ]:
class RNNgenerator_2(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru1 = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.gru2 = tf.keras.layers.GRU(rnn_units,
                                    return_sequences=True,
                                    return_state=True,
                                    recurrent_initializer='glorot_uniform')
    self.gru3 = tf.keras.layers.GRU(rnn_units,
                                    return_sequences=True,
                                    return_state=True,
                                    recurrent_initializer='glorot_uniform')
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, x, states_1=None, states_2=None, states_3=None, return_state=False):

    x = self.embedding(x)

    if states is None:
      states_1 = self.gru1.get_initial_state(x)
      states_2 = self.gru1.get_initial_state(x)
      states_3 = self.gru1.get_initial_state(x)

    x, states_1 = self.gru1(x, initial_state=states_1)
    x, states_2 = self.gru2(x, initial_state=states_2)
    x, states_3 = self.gru3(x, initial_state=states_3)

    x = self.dense(x)

    if return_state:
      return x, states_1, states_2, states_3
    else:
      return x

In [ ]:
checkpoint_dir = './RNN_2_checkpoints'
checkpoint_dir = './RNN_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                                filepath = checkpoint_prefix,
                                save_freq = 1,
                                save_weights_only = True)

In [ ]:
model_2 = RNNgenerator_2(
                 vocab_size = vocab_size,
                 embedding_dim = embedding_dim,
                 rnn_units = rnn_units,
                )

In [ ]:
model_2.compile(optimizer = 'adam', loss = loss, metrics = 'accuracy')

In [ ]:
model_2.load_weights('./RNN_checkpoints/ckpt_200')

In [ ]:
def generate_text_2(model, start_string, states_1, states_2, states_3, temperature):
    # Evaluation step (generating text using the learned model)

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions, states_1,states_2, states_3 = model(input_eval, states_1=states_1, states_2=states_2, states_3=states_3, return_state=True)
        predictions = tf.squeeze(predictions, 0)
        
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [ ]:
num_generate = 500
temperature = .5
states_1 = None
states_2 = None
states_3 = None
next_char = u"И вот идет уже "
text_generated = []

---

In [ ]:
text = generate_text_2(model_2, next_char, states_1, states_2, states_3, temperature)
print(text)